<a href="https://colab.research.google.com/github/uthours/Data-Processing-Visualization/blob/main/uttamDPVquiz5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SQL Queries
# Dr. Jacob Data Processing Visualization 
# Uttam Subedi

I am going to import the libraies. I am going to be wroking based on previous assignments.

In [19]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Now that I am authenticated, I can start to play around in the dataset. I am going to look at the Austin bikeshare trips data from Texas and calculate the second most popular starting station? And How many trips lasted over an hour and were a round trip (started and stopped at the same station? I am going to look at the bikers trip data from Austin. Yes, I have a project called fabled-progress-338823 in my BigQuery interface.

# The number of entries in data base:

In [20]:
%%bigquery --project fabled-progress-338823
SELECT 
  COUNT(*) as total_entries
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`

,total_entries
0,1424786


# Let's find the longest trip

In [21]:
%%bigquery --project fabled-progress-338823
SELECT 
  MAX(duration_minutes) as Longest_trip_in_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`

,Longest_trip_in_minutes
0,34238


# Now, let;s find the average time for a trip based on starting point

In [22]:
%%bigquery --project fabled-progress-338823
SELECT 
  start_station_name, 
  AVG(duration_minutes) as Average_time_for_trip
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name is not null
GROUP BY start_station_name

,start_station_name,Average_time_for_trip
0,Zilker Park West,27.168259
1,Toomey Rd @ South Lamar,28.115798
2,State Capitol @ 14th & Colorado,30.421842
3,Waller & 6th St.,23.265843
4,Pease Park,28.648889
...,...,...
188,Republic Square @ Guadalupe & 4th St.,23.067313
189,3rd & West,19.937466
190,3rd/West,34.152547
191,Nueces & 26th,16.805353


# I am going to compute the number of trips based on starting point.

In [23]:
%%bigquery --project fabled-progress-338823
SELECT 
  start_station_name,
  COUNT(*) as total_trips
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
ORDER BY total_trips DESC

,start_station_name,total_trips
0,21st & Speedway @PCL,72799
1,Riverside @ S. Lamar,40635
2,City Hall / Lavaca & 2nd,36520
3,2nd & Congress,35307
4,Rainey St @ Cummings,34758
...,...,...
188,Marketing Event,4
189,Eeyore's 2018,2
190,Stolen,1
191,Eeyore's 2017,1


# Let's find the second most popular starting station

In [24]:
%%bigquery --project fabled-progress-338823

SELECT COUNT(*) as total_trips, start_station_name
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
ORDER BY total_trips DESC

,total_trips,start_station_name
0,72799,21st & Speedway @PCL
1,40635,Riverside @ S. Lamar
2,36520,City Hall / Lavaca & 2nd
3,35307,2nd & Congress
4,34758,Rainey St @ Cummings
...,...,...
188,4,Marketing Event
189,2,Eeyore's 2018
190,1,Stolen
191,1,Eeyore's 2017


# Above table shows total trips started from each stations

In [25]:
%%bigquery --project fabled-progress-338823
SELECT MAX(pop_trip) as pop_trip
FROM (
SELECT COUNT(*) as pop_trip, start_station_name
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
)


,pop_trip
0,72799


# Now I will find the second most popular trip

In [26]:
%%bigquery --project fabled-progress-338823
SELECT start_station_name, COUNT(*) as most_pop_trip
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
HAVING most_pop_trip!= (SELECT MAX(most_pop_trip)as max_trip
FROM (
SELECT COUNT(*) as most_pop_trip, start_station_name
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
))
ORDER BY most_pop_trip DESC
LIMIT 1

,start_station_name,most_pop_trip
0,Riverside @ S. Lamar,40635


# ### Computing the trips lasted over an hour and were a round trip

In [27]:
%%bigquery --project fabled-progress-338823

SELECT start_station_name, duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = end_station_name

,start_station_name,duration_minutes
0,Toomey Rd @ South Lamar,82
1,Waller & 6th St.,76
2,Waller & 6th St.,75
3,Waller & 6th St.,74
4,Waller & 6th St.,73
...,...,...
228334,3rd/West,63
228335,3rd & West,63
228336,3rd/West,63
228337,3rd/West,63


In [28]:
%%bigquery --project fabled-progress-338823

SELECT *, COUNTIF(duration_minutes>60) as number_of_trip_took_over_an_hour
FROM (
  SELECT start_station_name, duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = end_station_name
)
GROUP BY duration_minutes, start_station_name
Order BY number_of_trip_took_over_an_hour DESC

,start_station_name,duration_minutes,number_of_trip_took_over_an_hour
0,Riverside @ S. Lamar,62,109
1,Riverside @ S. Lamar,61,95
2,Riverside @ S. Lamar,63,94
3,Riverside @ S. Lamar,69,90
4,Zilker Park,61,88
...,...,...,...
24822,26th/Nueces,40,0
24823,26th/Nueces,17,0
24824,26th/Nueces,53,0
24825,26th/Nueces,48,0
